In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pylast
import json

In [2]:
raw_features = []
for i in range(100):
    file = pd.read_csv("Spotify_API/dfs_first_100_playlists_features/df_playlist_features_"+str(i)+".csv")
    raw_features.append(file)


Last.fm tags into response labels

In [3]:

#unique tags threshold
threshold = 3000
#threshold = 1000
unique_tags = []
with open("last.fm/lastfm_unique_tags.txt", 'r') as f:
    for line in f:
        tag_freq = line.split('\t')
        if int(tag_freq[1]) > threshold:
            unique_tags.append(tag_freq[0])
        else:
            break

In [4]:
#create all zero value dictionary

def get_zero_dict():
    zeros_dict = {}
    for tag in unique_tags:
        zeros_dict.update({tag: 0.0})
    return zeros_dict

take the mean value of features of all the songs in the playlist

In [5]:
feature_names = list(raw_features[0].columns)[4:]
#print(feature_names)

labels = []
with open('labels_100_playlists.json') as json_file:
    data = json.load(json_file)
    label_columns = list(data[0].keys())
    for item in data:
        #label = list(item.values())
        #label = np.array(label)*100
        #labels.append(list(label))
        labels.append(list(item.values()))
    
features = []
for raw_playlist in raw_features:
    raw_playlist = raw_playlist.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    new_feature_array_playlist = list(raw_playlist.mean())
    features.append(new_feature_array_playlist)

overall_data = []
for i in range(len(features)):
    overall_data.append(features[i]+labels[i])

overall_df = pd.DataFrame(overall_data, columns = feature_names+label_columns)
overall_df.head()

,acousticness,danceability,speechiness,instrumentalness,liveness,energy,valence,duration_ms,key,mode,...,World Music,genius,noise,saxophone,Favorite Artists,indietronica,feelgood,new,folk-rock,pretty
0,0.085044,0.663922,0.107875,0.000689,0.192404,0.779078,0.645608,222076.627451,5.117647,0.705882,...,0.001154,0.000385,0.0000,0.0,0.000962,0.0000,0.001731,0.001346,0.0,0.000577
1,0.166732,0.493655,0.089239,0.228118,0.174761,0.686868,0.472711,299176.157895,4.578947,0.552632,...,0.000000,0.000000,0.0000,0.0,0.000000,0.0000,0.000000,0.000256,0.0,0.000000
2,0.273359,0.671952,0.096359,0.000648,0.170416,0.690095,0.565460,219076.174603,4.984127,0.507937,...,0.000000,0.000000,0.0000,0.0,0.000000,0.0000,0.000000,0.000000,0.0,0.007812
3,0.266913,0.509253,0.066359,0.210820,0.197497,0.624924,0.460534,229373.898990,5.090909,0.707071,...,0.000000,0.000200,0.0075,0.0,0.002800,0.0165,0.001400,0.000200,0.0,0.001700
4,0.181532,0.582188,0.041513,0.086992,0.169556,0.636631,0.489375,256603.562500,3.187500,0.812500,...,0.000000,0.000000,0.0000,0.0,0.000000,0.0000,0.000000,0.000000,0.0,0.000000


When find top n labels and label them with 1 (others 0).

In [6]:
feature_names = list(raw_features[0].columns)[4:]
#print(feature_names)

top_n_tags = 20

labels = []
with open('labels_100_playlists.json') as json_file:
    data = json.load(json_file)
    label_columns = list(data[0].keys())
    for item in data:
        label = list(item.values())
        sorted_label_top_n = sorted(label, reverse=True)[0:top_n_tags]
        new_label = [0]*len(label)
        for num in sorted_label_top_n:
            new_label[label.index(num)] = 1
        
        labels.append(new_label)
        #labels.append(list(item.values()))
    
features = []
for raw_playlist in raw_features:
    raw_playlist = raw_playlist.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    new_feature_array_playlist = list(raw_playlist.mean())
    features.append(new_feature_array_playlist)

overall_data = []
for i in range(len(features)):
    overall_data.append(features[i]+labels[i])

overall_df = pd.DataFrame(overall_data, columns = feature_names+label_columns)
overall_df.head()

,acousticness,danceability,speechiness,instrumentalness,liveness,energy,valence,duration_ms,key,mode,...,World Music,genius,noise,saxophone,Favorite Artists,indietronica,feelgood,new,folk-rock,pretty
0,0.085044,0.663922,0.107875,0.000689,0.192404,0.779078,0.645608,222076.627451,5.117647,0.705882,...,0,0,0,0,0,0,0,0,0,0
1,0.166732,0.493655,0.089239,0.228118,0.174761,0.686868,0.472711,299176.157895,4.578947,0.552632,...,0,0,0,0,0,0,0,0,0,0
2,0.273359,0.671952,0.096359,0.000648,0.170416,0.690095,0.565460,219076.174603,4.984127,0.507937,...,0,0,0,0,0,0,0,0,0,1
3,0.266913,0.509253,0.066359,0.210820,0.197497,0.624924,0.460534,229373.898990,5.090909,0.707071,...,0,0,0,0,0,0,0,0,0,0
4,0.181532,0.582188,0.041513,0.086992,0.169556,0.636631,0.489375,256603.562500,3.187500,0.812500,...,0,0,0,0,0,0,0,0,0,0


Train-test split

In [7]:
from sklearn.model_selection import train_test_split

#standardize (normalize X_df)
X_df = overall_df[feature_names]
X_df=(X_df-X_df.mean())/X_df.std()
y_df = overall_df[label_columns]

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2)


ANN model

In [8]:
# ouput activation: linear
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, input_dim=13, activation='relu'),
  tf.keras.layers.Dense(40, input_dim = 20, activation = 'linear'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(274, activation='linear')
])

# compile the keras model
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=300, batch_size=20, verbose=0)

_,train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print('Training Accuracy: %.2f' % (train_accuracy*100))

_,test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.2f' % (test_accuracy*100))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training Accuracy: 5.00
Test Accuracy: 15.00


In [9]:
# ouput activation: sigmoid
model_sigmoid_with_linear = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, input_dim=13, activation='relu'),
  tf.keras.layers.Dense(40, input_dim = 20, activation = 'linear'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(274, activation='sigmoid')
])

# compile the keras model
model_sigmoid_with_linear.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_sigmoid_with_linear.fit(X_train, y_train, epochs=300, batch_size=20, verbose=0)

_,train_accuracy = model_sigmoid_with_linear.evaluate(X_train, y_train, verbose=0)
print('Training Accuracy: %.2f' % (train_accuracy*100))

_,test_accuracy = model_sigmoid_with_linear.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.2f' % (test_accuracy*100))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Training Accuracy: 97.57
Test Accuracy: 94.42


In [10]:
probs_sigmoid_with_linear = model_sigmoid_with_linear.predict(X_test)
probs_sigmoid_with_linear

array([[6.6373187e-01, 9.9847394e-01, 6.1939943e-01, ..., 1.8805265e-05,
        1.6421080e-05, 1.4381397e-01],
       [4.1042763e-01, 9.9997324e-01, 8.4382403e-01, ..., 2.0861626e-07,
        2.0861626e-07, 2.8580487e-02],
       [6.3554579e-01, 6.9452047e-01, 1.8082467e-01, ..., 1.1920929e-07,
        5.9604645e-08, 5.3256750e-05],
       ...,
       [8.4524202e-01, 9.9967837e-01, 9.7104192e-01, ..., 0.0000000e+00,
        0.0000000e+00, 5.0663948e-07],
       [9.3135953e-01, 6.1041820e-01, 1.5199488e-01, ..., 3.0100346e-06,
        1.0818243e-05, 8.9406967e-08],
       [9.3865430e-01, 1.8767163e-01, 9.6377248e-01, ..., 1.6808510e-05,
        4.8279762e-06, 1.7881393e-07]], dtype=float32)

In [11]:
model_sigmoid = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, input_dim=13, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(274, activation='sigmoid')
])

# compile the keras model
model_sigmoid.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_sigmoid.fit(X_train, y_train, epochs=300, batch_size=20, verbose=0)

_,train_accuracy = model_sigmoid.evaluate(X_train, y_train, verbose=0)
print('Training Accuracy: %.2f' % (train_accuracy*100))

_,test_accuracy = model_sigmoid.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.2f' % (test_accuracy*100))

Training Accuracy: 96.69
Test Accuracy: 95.07


In [12]:
model_sigmoid.predict(X_test)

array([[5.8428270e-01, 9.8033547e-01, 4.9242327e-01, ..., 1.7479062e-04,
        1.6358495e-04, 1.3241139e-01],
       [3.9406994e-01, 9.9942052e-01, 8.4513140e-01, ..., 5.3942204e-06,
        6.6757202e-06, 5.3915590e-02],
       [8.3960295e-01, 2.2968027e-01, 9.6230775e-02, ..., 1.7583370e-06,
        2.2351742e-06, 1.6769767e-04],
       ...,
       [8.4513485e-01, 9.9672103e-01, 9.0637910e-01, ..., 8.9406967e-08,
        3.5762787e-07, 8.2105398e-05],
       [9.5967406e-01, 5.7292360e-01, 3.3851159e-01, ..., 4.1157007e-05,
        1.8656254e-05, 7.4505806e-07],
       [9.5869821e-01, 4.5909372e-01, 9.6265197e-01, ..., 1.6570091e-05,
        2.5451183e-05, 4.1723251e-07]], dtype=float32)

In [13]:
# ouput activation: sigmoid (loss_function: mean absolute error)
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, input_dim=13, activation='relu'),
  tf.keras.layers.Dense(40, input_dim = 20, activation = 'linear'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(274, activation='sigmoid')
])

# compile the keras model
model.compile(loss='mean_absolute_error', optimizer='adadelta', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=20, verbose=0)

_,train_accuracy = model.evaluate(X_train, y_train, verbose=1)
print('Training Accuracy: %.2f' % (train_accuracy*100))

_,test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy: %.2f' % (test_accuracy*100))

80/80 [==============================] - 0s 468us/sample - loss: 0.5003 - acc: 0.0000e+00
Training Accuracy: 0.00
20/20 [==============================] - 0s 27us/sample - loss: 0.5003 - acc: 0.0000e+00
Test Accuracy: 0.00


In [14]:
model.predict(X_test)

array([[0.41673705, 0.48534018, 0.54806006, ..., 0.5089556 , 0.46977597,
        0.5506634 ],
       [0.42541397, 0.46821764, 0.55730087, ..., 0.5031049 , 0.44335607,
        0.54143   ],
       [0.39183775, 0.4963807 , 0.58997333, ..., 0.49727148, 0.37965566,
        0.6536895 ],
       ...,
       [0.45275742, 0.5119368 , 0.57730836, ..., 0.48390085, 0.38033125,
        0.60264015],
       [0.5580218 , 0.5372651 , 0.5812974 , ..., 0.47726208, 0.40879658,
        0.64835674],
       [0.51903504, 0.5270925 , 0.5086614 , ..., 0.4947985 , 0.4452473 ,
        0.5590695 ]], dtype=float32)

CNN Model (Convoluted Neural Networks)

In [15]:
playlist_features = []
valid_data_indices = []
for i in range(0, 100):
    feature_sing = pd.read_csv("Spotify_API/dfs_first_100_playlists_features/df_playlist_features_" + str(i) + ".csv")
    feature_sing = feature_sing.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    feature_sing = feature_sing.dropna()
    feature_sing = feature_sing.to_numpy()
    num_songs = feature_sing.shape[0]
    if (num_songs >= 30):
        valid_data_indices.append(i)
        playlist_features.append(feature_sing[:30, ].reshape((30, 13, 1)))
playlist_features = np.array(playlist_features)
print(len(valid_data_indices))
playlist_features.shape

66


(66, 30, 13, 1)

In [16]:
# standardize features
playlist_features_mean = np.mean(playlist_features, axis=(0, 1), keepdims = True)
playlist_features_std = np.std(playlist_features, axis=(0, 1), keepdims = True)
playlist_features = (playlist_features - playlist_features_mean) / playlist_features_std

In [17]:
# read in labels
playlist_tags = []
with open('labels_100_playlists.json') as json_file:
    raw_label = json.load(json_file)
    for i in range(len(raw_label)):
        # filter labels with valid features
        if i in valid_data_indices:
            label = raw_label[i]
            label = np.array(list(label.values()))
            playlist_tags.append(label)

    playlist_tags = np.array(playlist_tags)

In [22]:
# train-test split get random indices
import random
from random import sample
random.seed( 209 )
test_indices = random.sample(range(0, 66), 7)

# split features
train_features = []
test_features = []
for i in range(len(playlist_features)):
    if i in test_indices:
        test_features.append(playlist_features[i])
    else:
        train_features.append(playlist_features[i])

train_features = np.array(train_features)
test_features = np.array(test_features)

# split labels
train_labels = []
test_labels = []
for i in range(len(playlist_tags)):
    if i in test_indices:
        test_labels.append(playlist_tags[i])
    else:
        train_labels.append(playlist_tags[i])

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [23]:
from tensorflow.keras import layers
from tensorflow.keras import models

input_shape = train_features[0].shape
num_category = train_labels[0].shape[0]

##model building
cnn_model = models.Sequential()
#convolutional layer with rectified linear unit activation
cnn_model.add(layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
#32 convolution filters used each of size 3x3
#again
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
cnn_model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
cnn_model.add(layers.Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
cnn_model.add(layers.Flatten())
#fully connected to get all relevant data
cnn_model.add(layers.Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
cnn_model.add(layers.Dropout(0.2))
#output a softmax to squash the matrix into output probabilities
cnn_model.add(layers.Dense(num_category, activation='sigmoid'))

In [24]:
# Use adaDelta as the adaptive learning rate
cnn_model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
                 metrics = ['accuracy'])

In [25]:
# Train Model
# too few data to have a batch_size
batch_size = 5
num_epoch = 30
model_log = cnn_model.fit(train_features, train_labels,
          epochs=num_epoch,
          verbose=1,
          validation_data=(test_features, test_labels))

Train on 59 samples, validate on 7 samples
Epoch 1/30
59/59 [==============================] - 0s 5ms/sample - loss: 0.6753 - acc: 0.2649 - val_loss: 0.5968 - val_acc: 0.3441
Epoch 2/30
59/59 [==============================] - 0s 656us/sample - loss: 0.5714 - acc: 0.3406 - val_loss: 0.4626 - val_acc: 0.3848
Epoch 3/30
59/59 [==============================] - 0s 680us/sample - loss: 0.4592 - acc: 0.3731 - val_loss: 0.3323 - val_acc: 0.4150
Epoch 4/30
59/59 [==============================] - 0s 607us/sample - loss: 0.3641 - acc: 0.4023 - val_loss: 0.2245 - val_acc: 0.4453
Epoch 5/30
59/59 [==============================] - 0s 618us/sample - loss: 0.2729 - acc: 0.4291 - val_loss: 0.1457 - val_acc: 0.4682
Epoch 6/30
59/59 [==============================] - 0s 647us/sample - loss: 0.2058 - acc: 0.4473 - val_loss: 0.0952 - val_acc: 0.4791
Epoch 7/30
59/59 [==============================] - 0s 670us/sample - loss: 0.1437 - acc: 0.4651 - val_loss: 0.0668 - val_acc: 0.4891
Epoch 8/30
59/59 [===

In [26]:
score = cnn_model.evaluate(train_features, train_labels, verbose=1)
print(score)

59/59 [==============================] - 0s 126us/sample - loss: 0.0417 - acc: 0.4853
[0.04170754343523818, 0.4853396]


In [27]:
cnn_model.predict(train_features)
cnn_test_predicted = cnn_model.predict(test_features)
cnn_test_predicted

array([[0.09066769, 0.12159464, 0.04062372, ..., 0.00159842, 0.00163046,
        0.00181997],
       [0.07123941, 0.09311324, 0.03540087, ..., 0.00096443, 0.0008665 ,
        0.00086579],
       [0.08004594, 0.12085637, 0.03253919, ..., 0.00108448, 0.00102615,
        0.00115463],
       ...,
       [0.1292525 , 0.16490254, 0.07589   , ..., 0.00544813, 0.00500983,
        0.00518134],
       [0.07063195, 0.09879074, 0.03174132, ..., 0.00076237, 0.00064006,
        0.00073072],
       [0.06935436, 0.09634471, 0.02801588, ..., 0.00060897, 0.00056572,
        0.00068578]], dtype=float32)

In [28]:
score = cnn_model.evaluate(test_features, test_labels, verbose=1)
print(score)

7/7 [==============================] - 0s 230us/sample - loss: 0.0377 - acc: 0.4896
[0.03769812732934952, 0.4895725]


Trees and Random Forests

In [29]:
feature_names = list(raw_features[0].columns)[4:]
#print(feature_names)

top_n_tags = 1

labels = []
with open('labels_100_playlists.json') as json_file:
    data = json.load(json_file)
    label_columns = list(data[0].keys())
    for item in data:
        label = list(item.values())
        sorted_label_top_n = sorted(label, reverse=True)[0:top_n_tags]
        new_label = [0]*len(label)
        for num in sorted_label_top_n:
            new_label[label.index(num)] = 1
        
        labels.append(new_label)
        #labels.append(list(item.values()))
    
features = []
for raw_playlist in raw_features:
    raw_playlist = raw_playlist.drop(columns=['track_name', 'artist', 'id','Unnamed: 0'])
    new_feature_array_playlist = list(raw_playlist.mean())
    features.append(new_feature_array_playlist)

overall_data = []
for i in range(len(features)):
    overall_data.append(features[i]+labels[i])

overall_df = pd.DataFrame(overall_data, columns = feature_names+label_columns)
overall_df.head()

,acousticness,danceability,speechiness,instrumentalness,liveness,energy,valence,duration_ms,key,mode,...,World Music,genius,noise,saxophone,Favorite Artists,indietronica,feelgood,new,folk-rock,pretty
0,0.085044,0.663922,0.107875,0.000689,0.192404,0.779078,0.645608,222076.627451,5.117647,0.705882,...,0,0,0,0,0,0,0,0,0,0
1,0.166732,0.493655,0.089239,0.228118,0.174761,0.686868,0.472711,299176.157895,4.578947,0.552632,...,0,0,0,0,0,0,0,0,0,0
2,0.273359,0.671952,0.096359,0.000648,0.170416,0.690095,0.565460,219076.174603,4.984127,0.507937,...,0,0,0,0,0,0,0,0,0,0
3,0.266913,0.509253,0.066359,0.210820,0.197497,0.624924,0.460534,229373.898990,5.090909,0.707071,...,0,0,0,0,0,0,0,0,0,0
4,0.181532,0.582188,0.041513,0.086992,0.169556,0.636631,0.489375,256603.562500,3.187500,0.812500,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from sklearn.model_selection import train_test_split

#standardize (normalize X_df)
X_df = overall_df[feature_names]
#X_df=(X_df-X_df.mean())/X_df.std()
y_df = overall_df[label_columns]

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2)


In [39]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(max_depth = 50, min_samples_split = 3, min_samples_leaf = 1,
                                   criterion = "gini").fit(X_train, y_train)
train_tree_acc = tree_model.score(X_train, y_train)
test_tree_acc = tree_model.score(X_test, y_test)
print("Training accuracy:", train_tree_acc)
print("Test accuracy:", test_tree_acc)

Training accuracy: 0.85
Test accuracy: 0.2


In [40]:
#tree_model.predict_proba(np.array([list(X_test.iloc[0])]))


In [41]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 100, criterion = "entropy",
                                            min_samples_split = 3, min_samples_leaf = 2,
                                            max_features = "sqrt").fit(X_train, y_train)
train_tree_acc = random_forest_model.score(X_train, y_train)
test_tree_acc = random_forest_model.score(X_test, y_test)
print("Training accuracy:", train_tree_acc)
print("Test accuracy:", test_tree_acc)

Training accuracy: 0.5625
Test accuracy: 0.1


In [87]:
#random_forest_model.classes_

Now with the probabilities, we can find use k nearest neighbors method to recommend songs for the user with specific playlist. Metric method 1: use the exact predicted probabilities and the normalized weights of the tags of the songs (by dividing 100), compute the mean squared difference. Metric method 2: classifiy the tags into 1 and 0 and compute the distance between the predicted tags and actual tags of the song.

random sampling million songs datasets files

In [ ]:
"""
playlists = []
random_indices = np.random.choice([i for i in range(1000)], size = 10)

# load data into pandas dataframe
for idx in random_indices:
    songs_df = pd.read_csv("songs/songs{}.csv".format(idx))

    # get playlists into dictionaries and store in a list

    id_nums = set(songs_df['pid'].tolist())
    songs_df = songs_df.groupby("pid")

    for id in id_nums:
        playlist = songs_df.get_group(id)
        playlists.append(playlist.to_dict())

# store the list of dictionaries into json file
with open('random_playlists', 'w') as fout:
    json.dump(playlists, fout)
"""

read in random_playlists json file and randomly sample n playlists from the json file

In [42]:
"""
import random

n = 100
with open('random_playlists.json') as json_file:
    data = json.load(json_file)
    random_data = random.sample(data, n)
    #print(len(random_data))
"""

concatenate pairs of artists and track names of each track into one list

In [44]:
"""
playlists_artist_tracks = []
for playlist in random_data:
    artist_names = list(playlist.get("artist_name").values())
    track_names = list(playlist.get("track_name").values())
    artist_tracks = zip(artist_names, track_names)
    playlists_artist_tracks += list(artist_tracks)
print(len(playlists_artist_tracks))
"""

'\nplaylists_artist_tracks = []\nfor playlist in random_data:\n    artist_names = list(playlist.get("artist_name").values())\n    track_names = list(playlist.get("track_name").values())\n    artist_tracks = zip(artist_names, track_names)\n    playlists_artist_tracks += list(artist_tracks)\nprint(len(playlists_artist_tracks))\n'

get the normalized tag weights of these randomly sampled songs

In [45]:
"""
songs_with_tags = {}

for track in playlists_artist_tracks:
    tag_dict = get_zero_dict()
    tags_with_weights = get_tags_from_track(track[0], track[1])
    if tags_with_weights is not None and len(tags_with_weights) != 0:
        for pair in tags_with_weights:
            tag_dict.update({pair[0]: pair[1]/100.0})
    print(tag_dict)
    songs_with_tags.update({(track[0], track[1]): list(tag_dict.values())})
"""

'\nsongs_with_tags = {}\n\nfor track in playlists_artist_tracks:\n    tag_dict = get_zero_dict()\n    tags_with_weights = get_tags_from_track(track[0], track[1])\n    if tags_with_weights is not None and len(tags_with_weights) != 0:\n        for pair in tags_with_weights:\n            tag_dict.update({pair[0]: pair[1]/100.0})\n    print(tag_dict)\n    songs_with_tags.update({(track[0], track[1]): list(tag_dict.values())})\n'

dump to json file

In [47]:
"""
candidates_with_tags = {}

for key_tuple in list(songs_with_tags.keys()):
    #turn tuple of artist_track into 'artist_name*,*track_name'
    candidates_with_tags.update({str(key_tuple[0])+'*,*'+str(key_tuple[1]): songs_with_tags.get(key_tuple)})

with open('candidates_with_tags', 'w') as fout:
    json.dump(candidates_with_tags, fout)
"""

"\ncandidates_with_tags = {}\n\nfor key_tuple in list(songs_with_tags.keys()):\n    #turn tuple of artist_track into 'artist_name*,*track_name'\n    candidates_with_tags.update({str(key_tuple[0])+'*,*'+str(key_tuple[1]): songs_with_tags.get(key_tuple)})\n\nwith open('candidates_with_tags', 'w') as fout:\n    json.dump(candidates_with_tags, fout)\n"

K-NN Song Recommender (Currently using k=1)

Songs Recommended by ANN Model

In [49]:
#read in json file as dictionary
with open('candidates_with_tags.json') as json_file:
    data = json.load(json_file)
songs_with_tags = data

In [53]:
# INPUT: tags for candidate songs, and the predicted tags of one playlist in the test dataset from a model
# OUTPUT: a song recommendation
def make_recommendation(candidate_tags, playlist_predicted_tag, random_songs):    
    min_distance = float('inf');
    rec = -1;
    for c in range(len(candidate_tags)):
        candidate = list(random_songs.values())[c]
        #each_distance = sum((np.array(candidate) - np.array(playlist_predicted_tag)) ** 2)
        each_distance = sum((candidate - playlist_predicted_tag) ** 2)
        if each_distance < min_distance:
            min_distance = each_distance;
            rec = c;
    return rec;

In [55]:
### Predict by KNN ### 
candidate_tags = list(songs_with_tags.keys())
recommendations = []

test_predicted = probs_sigmoid_with_linear

#playlists_artist_tracks can be replaced by accessing the dumped json file "candidates_with_tags"
playlists_artist_tracks = songs_with_tags
random_songs = playlists_artist_tracks


for each_playlist_predicted_tag in test_predicted:
    rec_for_test = make_recommendation(candidate_tags, each_playlist_predicted_tag, random_songs)
    recommendations.append(rec_for_test)
print(recommendations)

[2257, 5271, 3318, 5271, 3901, 2648, 3658, 4568, 236, 4670, 1383, 4119, 924, 888, 3318, 2257, 658, 5344, 1405, 241]


In [59]:
songs_recommended = [];
for rec in recommendations:
    rec_track = tuple(candidate_tags[rec].split("*,*"))
    songs_recommended.append(rec_track)
print(songs_recommended)

[('Jay Sean', 'Do You Remember'), ('Zara Larsson', 'Lush Life'), ('Snoop Dogg', 'Signs'), ('Zara Larsson', 'Lush Life'), ('Coldplay', 'Princess of China'), ('Daryl Hall & John Oates', 'Sara Smile'), ('Bon Iver', 'Skinny Love'), ('Rich Homie Quan', 'Flex (Ooh, Ooh, Ooh)'), ('The Flaming Lips', 'Fight Test'), ('Edwin McCain', "I'll Be"), ('Fountains Of Wayne', 'Hackensack'), ('WALK THE MOON', 'Shut Up and Dance'), ('Dave Matthews Band', 'Two Step'), ('Stealers Wheel', 'Stuck In The Middle With You'), ('Snoop Dogg', 'Signs'), ('Jay Sean', 'Do You Remember'), ('Paramore', 'Misguided Ghosts'), ('Beck', 'Wow'), ('David Bowie', 'Starman - 2012 Remastered Version'), ('The Killers', "All These Things That I've Done")]


Songs Recommended by CNN Model

In [60]:
### Predict by KNN ### 
candidate_tags = list(songs_with_tags.keys())
recommendations = []

test_predicted = cnn_test_predicted

#playlists_artist_tracks can be replaced by accessing the dumped json file "candidates_with_tags"
playlists_artist_tracks = songs_with_tags
random_songs = playlists_artist_tracks

for each_playlist_predicted_tag in test_predicted:
#     print(each_playlist_predicted_tag)
    rec_for_test = make_recommendation(candidate_tags, each_playlist_predicted_tag, random_songs)
    recommendations.append(rec_for_test)
print(recommendations)

[4, 4, 4, 4, 4567, 4, 4]


In [62]:
songs_recommended = [];
for rec in recommendations:
    rec_track = tuple(candidate_tags[rec].split("*,*"))
    songs_recommended.append(rec_track)
print(songs_recommended)

[('Disclosure', 'Omen - Radio Edit'), ('Disclosure', 'Omen - Radio Edit'), ('Disclosure', 'Omen - Radio Edit'), ('Disclosure', 'Omen - Radio Edit'), ('2 LIVE CREW', 'Mega Mixx Iii'), ('Disclosure', 'Omen - Radio Edit'), ('Disclosure', 'Omen - Radio Edit')]


For future K-NN Model use, we need to write a function to compute the k nearest songs

In [66]:
def distance(predicted, song):
    return np.sqrt(sum((np.array(predicted) - np.array(song)) ** 2))

# return the indices of the k nearest songs
def k_nearest_indices(predicted, songs, k):
    distances = []
    for song in songs:
        distances.append(distance(predicted, song))
    sorted_distances_top_k = sorted(distances)[0:k]
    k_indices = []
    for top_ele in sorted_distances_top_k:
        k_indices.append(distances.index(top_ele))
    return k_indices

# return the k nearest songs
def k_nearest_songs(predicted, songs_dict, k):
    songs = list(songs_dict.values())
    k_indices = k_nearest_indices(predicted, songs, k)
    k_recommended_songs = [tuple(list(songs_dict.keys())[idx].split("*,*")) for idx in k_indices]
    return k_recommended_songs
    

Demo for ANN Model

In [67]:
#read in json file as dictionary
with open('candidates_with_tags.json') as json_file:
    data = json.load(json_file)
songs_with_tags = data

In [68]:
songs_dict = songs_with_tags
test_predicted = probs_sigmoid_with_linear
for predicted in test_predicted:
    print(k_nearest_songs(predicted, songs_dict, 2))

[('Jay Sean', 'Do You Remember'), ('The Black Eyed Peas', 'Boom Boom Pow')]
[('Zara Larsson', 'Lush Life'), ('Dua Lipa', 'Hotter Than Hell')]
[('Snoop Dogg', 'Signs'), ('The Black Eyed Peas', 'Boom Boom Pow')]
[('Zara Larsson', 'Lush Life'), ('Dua Lipa', 'Hotter Than Hell')]
[('Coldplay', 'Princess of China'), ('Coldplay', 'Adventure Of A Lifetime')]
[('Daryl Hall & John Oates', 'Sara Smile'), ('Daryl Hall & John Oates', 'Rich Girl')]
[('Bon Iver', 'Skinny Love'), ('Paramore', 'Misguided Ghosts')]
[('Rich Homie Quan', 'Flex (Ooh, Ooh, Ooh)'), ('Beastie Boys', 'Brass Monkey')]
[('The Flaming Lips', 'Fight Test'), ('Grouplove', 'Ways To Go')]
[('Edwin McCain', "I'll Be"), ('Paramore', 'Misguided Ghosts')]
[('Fountains Of Wayne', 'Hackensack'), ('The White Stripes', "We're Going To Be Friends")]
[('WALK THE MOON', 'Shut Up and Dance'), ('Liz Phair', "Why Can't I?")]
[('Dave Matthews Band', 'Two Step'), ('OneRepublic', "Let's Hurt Tonight")]
[('Stealers Wheel', 'Stuck In The Middle With Yo

In [69]:
for predicted in test_predicted:
    print(k_nearest_songs(predicted, songs_dict, 3))

[('Jay Sean', 'Do You Remember'), ('The Black Eyed Peas', 'Boom Boom Pow'), ('Timbaland', 'The Way I Are')]
[('Zara Larsson', 'Lush Life'), ('Dua Lipa', 'Hotter Than Hell'), ('Marina and the Diamonds', 'How To Be A Heartbreaker')]
[('Snoop Dogg', 'Signs'), ('The Black Eyed Peas', 'Boom Boom Pow'), ('Missy Elliott', 'Get Ur Freak On')]
[('Zara Larsson', 'Lush Life'), ('Dua Lipa', 'Hotter Than Hell'), ('Twenty One Pilots', 'Heavydirtysoul')]
[('Coldplay', 'Princess of China'), ('Coldplay', 'Adventure Of A Lifetime'), ('Panic! At The Disco', 'That Green Gentleman (Things Have Changed)')]
[('Daryl Hall & John Oates', 'Sara Smile'), ('Daryl Hall & John Oates', 'Rich Girl'), ('The Jackson 5', 'I Want You Back')]
[('Bon Iver', 'Skinny Love'), ('Paramore', 'Misguided Ghosts'), ('Alexi Murdoch', 'Orange Sky')]
[('Rich Homie Quan', 'Flex (Ooh, Ooh, Ooh)'), ('Beastie Boys', 'Brass Monkey'), ('N.W.A.', 'Straight Outta Compton')]
[('The Flaming Lips', 'Fight Test'), ('Grouplove', 'Ways To Go'), ('P

The problem of this recommender system is that we are always recommending popular songs and failing to recommend songs of a niche or newly released songs. Since unpopular songs and newly released songs seldom have any tags or top frequent tags on the unique tags list. So we need to consider recommending such songs once in a while.